In [108]:
import os
import numpy as np
import torch
from PIL import Image
import pandas as pd

from project import *
from evaluation.metric import *

In [66]:
data_path = os.path.join(DATA_DIR, "market_bot")
DATASET_DIR = "/home/itsnamgyu/data/market1501"

In [8]:
qf = np.load(os.path.join(data_path, "qf.npy"))
gf = np.load(os.path.join(data_path, "gf.npy"))

In [22]:
qf = torch.Tensor(qf)
gf = torch.Tensor(gf)

In [28]:
distances = compute_distances(qf, gf)

In [31]:
qcamids = np.load(os.path.join(data_path, "qcamids.npy"))
gcamids = np.load(os.path.join(data_path, "gcamids.npy"))
qpids = np.load(os.path.join(data_path, "qpids.npy"))
gpids = np.load(os.path.join(data_path, "gpids.npy"))
qpaths = np.load(os.path.join(data_path, "qpaths.npy"))
gpaths = np.load(os.path.join(data_path, "gpaths.npy"))

In [34]:
qcamids = torch.Tensor(qcamids)
gcamids = torch.Tensor(gcamids)
qpids = torch.Tensor(qpids)
gpids = torch.Tensor(gpids)

In [50]:
distances = distances[:, gpids != -1]  # filter junk galleries (missing id)

In [102]:
person_overlap = (qpids.reshape(-1, 1) == gpids)
camera_overlap = (qcamids.reshape(-1, 1) == gcamids)
junk = camera_overlap & person_overlap

In [85]:
ranks = distances.argsort(dim=1)

In [86]:
ranks.shape

torch.Size([3368, 15913])

In [100]:
ranked_gp = gpids[ranks]

In [103]:
matches = (ranked_gp == qpids.reshape(-1, 1))

In [181]:
ranked_gp[728]

tensor([1174., 1067., 1067.,  ...,  625.,    0.,    0.])

In [186]:
last_ranks.shape

torch.Size([3368])

In [164]:
mean_ranks = (matches * torch.arange(0, g)).float().mean(dim=1)

In [176]:
torch.where(mean_ranks > 1)

(tensor([ 137,  184,  278,  359,  493,  580,  593,  630,  689,  728,  794,  832,
          938, 1021, 1205, 1334, 1341, 1393, 1559, 1811, 1897, 2196, 2416, 2479,
         2560, 2627, 2658, 2749, 2967, 3252]),)

In [188]:
torch.where(matches[137])

(tensor([   0,    1,    2,    3,    4,    5,    6,    8,   11,   12,   34,   35,
           36,   39,   46,   68,   73,  178,  185,  204,  461, 1090, 1553, 1645,
         2425, 5117, 5357]),)

In [123]:
def load_gallery_image(index):
    return Image.open(os.path.join(DATASET_DIR, gpaths[index]))

In [124]:
def load_query_image(index):
    return Image.open(os.path.join(DATASET_DIR, qpaths[index]))

In [25]:
query_index = np.argwhere(gl==ql)
camera_index = np.argwhere(gc==qc)

good_index = np.setdiff1d(query_index, camera_index, assume_unique=True)
junk_index1 = np.argwhere(gl==-1)
junk_index2 = np.intersect1d(query_index, camera_index)
junk_index = np.append(junk_index2, junk_index1) #.flatten())

tensor([[6.1203, 6.4899, 6.5363,  ..., 6.2125, 6.7233, 6.1606],
        [4.9783, 6.6089, 6.8476,  ..., 3.4175, 6.4988, 6.2843],
        [6.4559, 6.1467, 7.5682,  ..., 6.9706, 6.4142, 6.4559],
        ...,
        [7.6159, 5.5724, 5.7081,  ..., 9.4089, 6.1018, 6.1894],
        [7.1355, 5.4113, 5.2549,  ..., 8.5613, 5.1399, 4.6796],
        [6.6259, 5.8003, 5.3014,  ..., 8.9103, 5.6569, 6.3106]])